# 라이브러리 로드

In [1]:
import tqdm
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

from scipy.stats import skew 
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings(action='ignore')
plt.style.use('seaborn-whitegrid')
plt.rc('font', family='Malgun Gothic')  # 한글 폰트 사용을 위해서 세팅
pd.set_option('max_columns', 80) # 최대 컬럼 갯수 지정
pd.set_option('max_rows', 80) # 최대 로우 갯수 지정

# 데이터 로드 함수 정의

In [2]:
def load_data(data):
    ### 데이터를 로드하여 전처리하는 함수 ###
    df = pd.read_csv(data, encoding='cp949')
    df = df[df['상권_구분_코드'] == 'A']
    df['상권_코드'] = df['상권_코드'].astype('str')
    df['상권_코드'] = df['상권_코드'].apply(lambda x: str(x))
    df['서비스_업종_코드_명'] = df['서비스_업종_코드_명'].apply(service_recategorization)
    return df

def service_recategorization(x):
    ### 서비스 업종별 코드 전처리 ###
    if x == '스포츠클럽' or x == '운동/경기용품' or x == '노래방' or x == 'PC방' or x == '골프연습장' or x == '당구장' or x == '스포츠 강습':
        return '여가'
    elif x == '편의점' or x == '슈퍼마켓' or x == '세탁소' or x == '부동산중개업':
        return '편의시설'
    elif x == '청과상' or x == '수산물판매' or x == '미곡판매' or x == '육류판매':
        return '농수산물'
    elif x == '자동차수리' or x == '자동차미용' or x == '자전거 및 기타운송장비':
        return '운송장비'
    elif x == '컴퓨터및주변장치판매' or x == '가전제품' or x == '가전제품수리' or x == '핸드폰':
        return '가전'
    elif x == '가방' or x == '신발' or x == '일반의류' or x == '안경' or x == '섬유제품':
        return '의류'
    elif x == '예술학원' or x == '외국어학원' or x == '일반교습학원' or x == '문구' or x == '서적' or x == '고시원' :
        return '교육'
    elif x == '화장품' or x == '시계및귀금속' or x == '피부관리실' or x == '네일숍' or x == '미용실':
        return '미용'
    elif x == '치과의원' or x == '일반의원' or x == '의료기기' or x == '한의원' or x == '의약품':
        return '의료'
    elif x == '전자상거래업' or x == '여관' or x == '완구' or x == '철물점' or x == '화초' or x == '애완동물':
        return '기타'
    else:
        return '음식'

# 데이터 로드

##### row_data

In [3]:
store_2021 = load_data('./data/row_data/서울시 우리마을가게 상권분석서비스(상권-추정매출).csv') 
store_2020 = load_data('./data/row_data/서울시 우리마을가게상권분석서비스(상권-추정매출)_2020.csv') 
store_2019 = load_data('./data/row_data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv')
store_2018 = load_data('./data/row_data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv')

In [4]:
store_2021.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
11650,2021,2,A,골목상권,1001010,풍성로37가길,CS300043,기타,1177203,58,100,0,47,48,0,5,0,0,0,0,0,22,24,54,0,55,45,0,0,7,93,0,0,1177203,0,555555,560345,0,61303,0,0,0,0,0,252874,287356,636973,0,651341,525862,0,0,81418,1095785,0,0,58,0,19,29,0,10,0,0,0,0,0,19,10,29,0,19,39,0,0,10,48,0,0,5
11651,2021,2,A,골목상권,1001010,풍성로37가길,CS300033,기타,7277987,878,81,19,31,5,26,10,9,7,12,0,34,23,35,9,0,78,22,0,8,7,19,27,39,5916005,1361982,2242880,390067,1901572,715122,666364,487583,874399,0,2486673,1657782,2515928,617604,0,3523599,991419,0,357561,292550,861397,1235210,1768301,714,164,257,65,131,130,131,66,98,0,228,228,260,162,0,520,196,0,98,33,163,196,228,2
11652,2021,2,A,골목상권,1001010,풍성로37가길,CS300018,의료,2073726436,120743,90,11,21,16,17,18,18,11,0,0,23,32,35,10,0,51,49,0,2,9,15,20,53,1856780239,216946197,425426207,333894894,354270423,374768421,368420294,216946197,0,0,474389984,665135646,725766440,208434366,0,1050929335,993478864,417374,45731492,191294411,304330562,415140991,1087493369,110299,10444,24293,21082,19073,22234,23617,10444,0,0,30645,33852,41975,14271,0,62650,57702,61,6010,12898,20336,25419,55627,3
11653,2021,2,A,골목상권,1001010,풍성로37가길,CS300016,의류,103383074,884,90,10,13,33,14,7,24,10,0,0,28,24,22,26,0,56,44,0,1,5,11,28,56,92728757,10654317,13342109,33670063,13983791,7203771,24529023,10654317,0,0,28536619,25146488,22882568,26817399,0,53562036,41527622,0,968574,4782336,9988422,26308777,53041549,763,121,195,169,133,108,158,121,0,0,243,279,278,84,0,508,351,0,12,60,169,266,352,1
11654,2021,2,A,골목상권,1001010,풍성로37가길,CS300010,음식,383132413,23232,82,18,18,17,17,16,15,18,0,0,31,18,28,24,0,49,51,0,0,17,10,37,37,315606133,67526280,68235385,64149372,64614500,60403137,58203739,67526280,0,0,118042837,69566957,105350491,90050715,121413,188257159,194875254,0,347135,65633558,36953365,139980535,140217820,19062,4170,3710,4017,3829,3497,4009,4170,0,0,4499,3955,7017,7753,8,11309,11923,0,102,2598,2447,7742,10343,5


##### base df

In [5]:
def get_object_type(df):
    df['상권_코드'] = df['상권_코드'].apply(lambda x : str(x))
    df.astype({'상권_코드':object})
    try:
        df.drop('상권_이름', inplace=True, axis=1)
    except:
        pass
    return df

bs_area = gpd.read_file('data/geojson/상권_geometry_좌표.geojson', driver= 'GeoJSON')
bs_area= bs_area[bs_area['TRDAR_SE_C'] == 'A']
df_base = bs_area[['TRDAR_CD','TRDAR_SE_1']].sort_values('TRDAR_CD').reset_index(drop=True)
df_base.columns = ['상권_코드', '상권_이름']
df_base = get_object_type(df_base)
df_base = df_base.drop(804, axis=0) # 없는 상권
print(df_base.shape)
df_base.to_csv('./data/base_data/df_base.csv', index=False)
df_base.head()

(1009, 1)


,상권_코드
0,1000001
1,1000002
2,1000003
3,1000004
4,1000005


## 테이블 Merge 함수 정의

In [6]:
# 분기당 매출 금액을 기준으로 비율을 구하는 함수
def divide_per_cat(df,col):
    tmp_df = df.groupby(['상권_코드', '기준_분기_코드'])[col].sum().to_frame()
    tmp_df.columns = [col]
    
    return tmp_df

def merge_on_table(df_2021, df_2020, df_2019, df_2018, col):
    df_2021 = divide_per_cat( df_2021, col )
    df_2021 = df_2021.unstack()
    df_2021.columns = ['2021년 1분기', '2021년 2분기']
    
    df_2020 = divide_per_cat( store_2020, col )
    df_2020 = df_2020.unstack()
    df_2020.columns = ['2020년 1분기', '2020년 2분기',
                       '2020년 3분기', '2020년 4분기']
    
    df_2019 = divide_per_cat( store_2019, col )
    df_2019 = df_2019.unstack()
    df_2019.columns = ['2019년 1분기', '2019년 2분기',
                       '2019년 3분기', '2019년 4분기']
    
    df_2018 = divide_per_cat( store_2018, col )
    df_2018 = df_2018.unstack()
    df_2018.columns = ['2018년 1분기', '2018년 2분기',
                       '2018년 3분기', '2018년 4분기']
    
    df = pd.merge(df_base, df_2018, on='상권_코드')
    df = pd.merge(df, df_2019, on='상권_코드')
    df = pd.merge(df, df_2020, on='상권_코드')
    df = pd.merge(df, df_2021, on='상권_코드')
    return df

## coefficient 반환 함수 정의

In [7]:
def making_coeff(df, index):
    lr = LinearRegression()
    lr.fit(np.arange(1, len(df.iloc[index,2:].values)+1).reshape(-1, 1), df.iloc[index,2:].values)
    coeff=lr.coef_
    return coeff

# 매출 금액 회귀 계수 도출

In [8]:
revenue = merge_on_table(store_2021, store_2020, store_2019, store_2018, '분기당_매출_금액')
revenue.head()

,상권_코드,2018년 1분기,2018년 2분기,2018년 3분기,2018년 4분기,2019년 1분기,2019년 2분기,2019년 3분기,2019년 4분기,2020년 1분기,2020년 2분기,2020년 3분기,2020년 4분기,2021년 1분기,2021년 2분기
0,1000001,3844094552,5037827298,4607094548,7043871460,3.373980e+09,3.329090e+09,3.751288e+09,6.001523e+09,5.560254e+09,4.046282e+09,3.330855e+09,3.289855e+09,3560490364,4945371583
1,1000002,6038145163,9178573434,7952738778,7285140367,5.786105e+09,4.914289e+09,5.331139e+09,5.586517e+09,3.068686e+09,4.458268e+09,3.386368e+09,4.630761e+09,3219012417,4758520142
2,1000003,10820147790,12523971659,9844910289,11041145402,8.605367e+09,7.510965e+09,8.622047e+09,8.645871e+09,7.720079e+09,6.801630e+09,6.955296e+09,6.869209e+09,7390675982,7528112723
3,1000004,4454301424,5291488602,4286924277,4557499843,2.973229e+09,3.496874e+09,2.945884e+09,3.067755e+09,2.342075e+09,2.703370e+09,2.813943e+09,2.669599e+09,2465025739,2773489996
4,1000005,3478761309,4065446266,4884620171,5598591748,2.233832e+09,2.273201e+09,2.947653e+09,3.161904e+09,2.789946e+09,3.481229e+09,2.753574e+09,2.669249e+09,2112848259,2899075340


In [9]:
coef_list = np.array([])

for i in range(revenue.shape[0]):
    coef = making_coeff(revenue, i)
    coef_list=np.append(coef_list, coef)
    
revenue['coef']=coef_list
print(revenue.shape)
revenue.head()

(1009, 16)


,상권_코드,2018년 1분기,2018년 2분기,2018년 3분기,2018년 4분기,2019년 1분기,2019년 2분기,2019년 3분기,2019년 4분기,2020년 1분기,2020년 2분기,2020년 3분기,2020년 4분기,2021년 1분기,2021년 2분기,coef
0,1000001,3844094552,5037827298,4607094548,7043871460,3.373980e+09,3.329090e+09,3.751288e+09,6.001523e+09,5.560254e+09,4.046282e+09,3.330855e+09,3.289855e+09,3560490364,4945371583,-9.719695e+07
1,1000002,6038145163,9178573434,7952738778,7285140367,5.786105e+09,4.914289e+09,5.331139e+09,5.586517e+09,3.068686e+09,4.458268e+09,3.386368e+09,4.630761e+09,3219012417,4758520142,-3.910998e+08
2,1000003,10820147790,12523971659,9844910289,11041145402,8.605367e+09,7.510965e+09,8.622047e+09,8.645871e+09,7.720079e+09,6.801630e+09,6.955296e+09,6.869209e+09,7390675982,7528112723,-3.637633e+08
3,1000004,4454301424,5291488602,4286924277,4557499843,2.973229e+09,3.496874e+09,2.945884e+09,3.067755e+09,2.342075e+09,2.703370e+09,2.813943e+09,2.669599e+09,2465025739,2773489996,-1.892185e+08
4,1000005,3478761309,4065446266,4884620171,5598591748,2.233832e+09,2.273201e+09,2.947653e+09,3.161904e+09,2.789946e+09,3.481229e+09,2.753574e+09,2.669249e+09,2112848259,2899075340,-1.580049e+08


In [10]:
revenue[['상권_코드','coef']].to_csv('./data/coeff/18~21_총_매출_회귀계수.csv', index=False)

In [11]:
pd.read_csv('./data/coeff/18~21_총_매출_회귀계수.csv').head()

,상권_코드,coef
0,1000001,-9.719695e+07
1,1000002,-3.910998e+08
2,1000003,-3.637633e+08
3,1000004,-1.892185e+08
4,1000005,-1.580049e+08
